In [1]:
import os
import sys
import urllib.request
import requests
import json
import logging
from selenium.webdriver import Chrome
import time
import glob
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
import bs4
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import re
import pyautogui
import csv
import datetime
from dateutil.relativedelta import relativedelta
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime as dt
from gensim.summarization.summarizer import summarize
from konlpy.tag import Mecab
from gensim.summarization import keywords
from collections import Counter
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
from collections import defaultdict

In [2]:
client_id = "api key"
client_secret = "api비밀번호"

In [3]:
if __name__ == '__main__':


    logger = logging.getLogger()

    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
       
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

In [ ]:
def get_naver_store(query,num=5):
    temp = 0
    page = 0
    naver_store = {}
    while num >0:
        time.sleep(2)
        if num>100:
            num = num - 100
            temp = 100
            page = page + 1
        else:
            temp = num
            num = 0
            page = page + 1

        logger.info("네이버 상점 크롤링")
        res = requests.get("https://map.naver.com/v5/api/search?caller=pcweb&query={}&type=all&page={}&displayCount={}&isPlaceRecommendationReplace=true&lang=ko".format(query,page,str(temp))).text        
        html = json.loads(res)

        for store in html['result']['place']['list'][:temp]:
            pos = store['id']
            if pos not in naver_store:
                naver_store[pos]= {}
            attrs = ['name','category','address','roadAddress','menuInfo','bizhourInfo']
            for attr in attrs:
                if attr in store:
                    naver_store[pos][attr]=store[attr]

    return naver_store
            
    
def get_naver_review(businessid,num=10):
    logger.info("네이버 리뷰 크롤링")
    data = { "query" }
    stores = {}
    user_list = []
    for idx,Id in enumerate(businessid):
        #Id에 가게정보 입력
        stores[Id]={}
        time.sleep(5)
        data['variables']['input']['businessId'] = Id
        reviews = json.loads(requests.post('https://pcmap-api.place.naver.com/graphql',json = data).text,encoding="UTF-8")
        try:
            for review in reviews['data']['visitorReviews']['items']:
                stores[Id][review['id']]={}
                stores[Id][review['id']]['rating'] = review['rating']
                stores[Id][review['id']]['nickname'] = review['author']['id']
                stores[Id][review['id']]['avgRating'] = review['author']['review']['avgRating']
                stores[Id][review['id']]['body'] = review['body']
                stores[Id][review['id']]['created'] = review['created']
                user_list.append({Id:review['author']['id']})
        except Exception as e:
            logger.error("get_naver_review 오류발생: {}".format(e))
            
    return stores, user_list


def date_reg(text):
    pattern = r"([0-9])+\.([0-9])+\.([월화수목금토일]+)"
    if re.match(pattern,text):
        return str(datetime.datetime.now().year)+"."+text
    else:
        return text
    
            
def get_user_review(userList,relation):
    logger.info("네이버 유저 리뷰 크롤링")
    data = { "query" }
    answer = []
    for idx, user in enumerate(userList):
        time.sleep(2)
        
        
        data['variables']['authorId'] = list(user.values())[0]
        data['variables']['limit'] = relation
        try:
            user_review = json.loads(requests.post('https://m.place.naver.com/my/graphql',json = data).text)
            for review in user_review['data']['authorReviews']['reviews'][:relation]:
                try:

                    temp = []
                    
                    #사용자가 작성한 첫 가게 비지니스 ID
                    temp.append(list(user.keys())[0])
                    
                    #사용자 닉네임
                    temp.append(list(user.values())[0])
                    
                    #사용자가 작성한 다른 가게 비지니스 ID
                    temp.append(review['place']['id'])
                    
                    #가게명
                    temp.append(review['place']['name'])
                    
                    #가게 주소
                    temp.append(review['place']['address'])
                    
                    #가게 평점
                    temp.append(review['rating'])
                    
                    #평점올린 날
                    temp.append(date_reg(review['visitedDate']['displayDate']))
                    
                except Exception as e:
                    logger.error("get_user_review 오류발생: {}".format(e))

                answer.append(temp)
        except Exception as e:
            logger.error("get_user_review request 보내는 과정에서 오류발생: {}".format(e))
            
    return answer
        
    

# [['35303633', '광교 라벤치', '경기도 수원시 영통구'], ['1026282129', '남도예찬', '경기도 수원시 영통구']]
def get_google_store(browser,naver_id,businessId,names,reviewNum):
    #naver businessId와, 가게 이름, 주소, 몇개의 리뷰를 가져올지
    logger.info("구글 가게 정보 가져오자!")
    sub = "https://www.google.com/search?q="
    result=[]
    
    #후기는 팝업이라 selenium으로 조정이 안됨.
    #pyautogui를 사용하여 마우스 조작으로 리뷰 로딩 본인 컴퓨터에 팝업이 나타나는 x,y 좌표 설정
    #pyautogui 참조
    x = 938
    y = 478
    
    #names 가게 리스트
    #headers = [naver_businessId, 가게이름, 가게주소]
    
    second_url = []
    user_id = []
    for idx2, name in enumerate(names):
        
        url = name[1] + " " + name[2]
        

        #url 에 바로 경기도 수원시 팔달구 우만동 중부대로223번길 41 본수원갈비 입력
        browser.get(sub+url)
        WebDriverWait(browser,5)
        time.sleep(1)
        
        try:
            #후기 클릭
            browser.find_element_by_class_name('hqzQac').click()
            time.sleep(2)
            #마우스 휠 내리기
            pyautogui.moveTo(x,y)

            pyautogui.click()
            pos = len(browser.find_elements_by_class_name('TSUbDb'))
            for i in range(100):
                pyautogui.scroll(-60)
                time.sleep(1)
                
                if len(browser.find_elements_by_class_name('TSUbDb')) >reviewNum:break
                    
                if pos == len(browser.find_elements_by_class_name('TSUbDb')): break
                else: pos = len(browser.find_elements_by_class_name('TSUbDb'))
                    
                    

                
            #사용자 아이디 가져오기
            ideas = browser.find_elements_by_class_name('TSUbDb')
            
            #별점에 대한 크롤링  
            scores = browser.find_elements_by_class_name('PuaHbe')
            
            #후기글에 대한 크롤링
            texts = browser.find_elements_by_class_name('Jtu6Td')
            
            #날짜정보
            dates = browser.find_elements_by_xpath("//span[@class='dehysf lTi8oc']")
            
            for idx,idea in enumerate(ideas[:reviewNum]):
                aaa = []
                aaa.append(idea.find_element(By.TAG_NAME,'a').get_attribute('href'))
                aaa.append(naver_id[idx2])
                second_url.append(aaa)
                
                temp = []
                temp.append(businessId[idx2])
                temp.append(name[1])
                temp.append(idea.text)
                
                score = google_score_reg(scores[idx].find_element(By.TAG_NAME,'span').get_attribute('aria-label'))
                temp.append(score)
                temp.append(texts[idx].text)
                
                date = day_week_year(dates[idx].text)
                temp.append(date)
                
                user_id.append(idea.text)
                result.append(temp)
        except Exception as e:
            logger.error("get_google_store 오류발생: {}".format(e))
        
    return result, second_url, user_id


def google_score_reg(rating):
    pattern = r"[0-9]+\.[0-9]+"
    if re.search(pattern,str(rating)):
        rating=re.search(pattern,str(rating))[0]
    return rating

def day_week_year(date):  
    current = datetime.datetime.now()
    if '주' in date:
        pos = date.find('주')
        pos = int(date[:pos])
        return current - datetime.timedelta(days=pos*7)
    elif '일' in date:
        pos = date.find('일')
        pos = int(date[:pos])
        return current - datetime.timedelta(days=pos)
    elif '달' in date:
        pos = date.find('달')
        pos = int(date[:pos])
        return current - relativedelta(months=pos)
    elif '년' in date:
        pos = date.find('년')
        pos = int(date[:pos])
        return current - relativedelta(years=pos)
    
    elif '시' in date:
        pos = date.find('시')
        pos = int(date[:pos])
        return current - datetime.timedelta(hours=pos)
    
    elif '분' in date:
        pos = date.find('분')
        pos = int(date[:pos])
        return current - datetime.timedelta(minutes=pos)    
            

def get_google_user_review(browser,userList,user_id,relation):
    answer = []
    bus_pos = 0
    for idx, user in enumerate(userList):
        
        browser.get(user[0])
        WebDriverWait(browser,5)
        time.sleep(1)
        try:
            browser.find_element_by_css_selector('[aria-label=리뷰]').click()
            time.sleep(1)
            pos = browser.find_element_by_class_name('section-tab-info').location
            pyautogui.moveTo(list(pos.values())[0]+200,list(pos.values())[1]+130)
            pyautogui.click()
            time.sleep(1)
            numberOfContent = len(browser.find_elements_by_class_name("section-review-content"))
            for i in range(100):
                pyautogui.scroll(-60)
                time.sleep(1)
                if len(browser.find_elements_by_class_name("section-review-content")) > relation: break

                if numberOfContent == len(browser.find_elements_by_class_name("section-review-content")): break
                else: numberOfContent = len(browser.find_elements_by_class_name("section-review-content"))
    
            titles = browser.find_elements_by_class_name('section-review-titles')
            stars = browser.find_elements_by_class_name('section-review-stars')
            dates = browser.find_elements_by_class_name('section-review-publish-date')

            for idx2, context in enumerate(titles[:relation]):
                temp = []
                number = 0
                name, address = context.text.split('\n')
                star = stars[idx2].find_elements(By.TAG_NAME,'img')

                for st in star:
                    if 'empyh' in st.get_attribute('src'):number = number + 1

                star = 5 - number

                #가게 이름, 가게 주소, 가게 별점, 별점 날짜,
                temp.append(user[1])
                temp.append(name)
                temp.append(user_id[idx])
                temp.append(address)
                temp.append(star)
                temp.append(day_week_year(dates[idx2].text))

                answer.append(temp)
        
        except Exception as e:
            logger.error("google_user_reivew: {}".format(e))
            
    return answer
        
def findStoreName(store_dict,businessId):
    logger.info("네이버 비지니스 id의 이름, 주소 가져오자!!")
    #store_dict: 네이버 스토어 정보를 담고 있는 store_dict
    #businessId: store_dict에서 찾을 store 리스트
    
    #using businessId find address and name
    result = []
    for businessid in businessId:
        add1 = None
        add2 = None
        temp = []
        if businessid in store_dict:
            try:
                name = store_dict[businessid]['name']
                if 'address' in store_dict[businessid]:
                    add1 = store_dict[businessid]['address']
                if 'roadAddress' in store_dict[businessid]:
                    add2 = store_dict[businessid]['roadAddress']
                temp.append(businessid)
                temp.append(name)
                if add1 is not None:
                    temp.append(reg(add1))
                elif add2 is not None:
                    temp.append(reg(add2))
                result.append(temp)
            except Exception as e:
                logger.error("findStoreName 오류 발생: {}".format(e))
    return result
                
                
def reg(name):
    pattern1 = r"[\S]+(도)\s[\S]+(시)\s[\S]+(구)"
    pattern2 = r"[\S]+(시)\s[\S]+(구)\s[\S]+(동)"
    pattern3 = r"[\S]+(도)\s[\S]+(시)\s[\S]+(시)\s[\S]+(동)"
    pattern4 = r"[\S]+(도)\s[\S]+(시)\s[\S]+(읍)"
    pattern5 = r"[\S]+(시)\s[\S]+(읍)"
    pattern6 = r"[\S]+(시)\s[\S]+(구)"
    pattern7 = r"[\S]+(시)\s[\S]+(군)"
    pattern9 = r"[\S]+(시)\s[\S]+(읍)"
    pattern8 = r"[\S]+(도)\s[\S]+(시)"
    pattern10 = r"[\S]+(도)\s[\S]+(군)"

    #지역
    pattern = []
    pattern.append(pattern1)
    pattern.append(pattern2)
    pattern.append(pattern3)
    pattern.append(pattern4)
    pattern.append(pattern5)
    pattern.append(pattern6)
    pattern.append(pattern7)
    pattern.append(pattern8)
    pattern.append(pattern9)
    pattern.append(pattern10)


    for a in pattern:
        if re.search(a,name) is not None: return re.search(a,name)[0]

def setSelenium(path='/home/com/다운로드/ch*'):
    #path에 chromedriver 주소를 입력해주세요.
    path = glob.glob(path)
    option = Options()

    path[0]

    browser = Chrome(executable_path=path[0], options=option)
    return browser , True
    
    
def save_data(dataframe):
    
    basename = ["naver_business", "naver_review", "naver_user_review", "google_review", "google_user_review"]
                
    for idx, df in enumerate(dataframe):
        suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
        filename = "-".join(['data/',basename[idx],suffix,'.csv'])
        df.to_csv(filename,header=df.columns)
        

        
def pipeLined(query,storeNum,reviewNum,relation,browser=None):
    #네이버에 검색할 query
    #가져올 업체의 개수
    #업체에 속한 리뷰의 개수
    #한 리뷰작성자가 이전에 평가한 리뷰들
    
    logger.info("크롤링 파이프라인 시작!!!")
    naver_store = get_naver_store(query,storeNum)
    stores, user_list = get_naver_review(naver_store,reviewNum)
    
    naver_user_review = get_user_review(user_list,relation)
    
    hasSelenium = False
    names = findStoreName(naver_store,list(naver_store))
    if browser is None:
        browser = setSelenium(hasSelenium)
    google_store, review_url, user_id = get_google_store(browser,list(stores) ,list(naver_store),names,reviewNum)
    
    google_user_review = get_google_user_review(browser,review_url,user_id,relation)
    
    
    
    
    store_to_list = []
    for store in stores:
        for st in stores[store]:
            temp = []
            temp.append(store)
            temp.append(stores[store][st]['rating'])
            temp.append(stores[store][st]['nickname'])
            temp.append(stores[store][st]['body'])
            temp.append(stores[store][st]['created'])
            store_to_list.append(temp)

    naver_store_list = []
    for naver in naver_store:
        temp = []
        temp.append(naver)
        temp.append(naver_store[naver]['name'])
        temp.append(naver_store[naver]['category'])
        temp.append(naver_store[naver]['address'])
        temp.append(naver_store[naver]['roadAddress'])
        temp.append(naver_store[naver]['menuInfo'])
        temp.append(naver_store[naver]['bizhourInfo'])
        naver_store_list.append(temp)
    
    df = []

    #네이버 비지니스 가게 정보
    df1 = pd.DataFrame(naver_store_list,columns = ['businessId','name','category','address','roadAddress','menuInfo','bizhourInfo'])

    #네이버 비지니스 가게의 후기 정보
    df2 = pd.DataFrame(store_to_list,columns = ['businessId','rating','nickname','body','created'])

    #네이버 개인과 관계된 가게정보
    df3 = pd.DataFrame(naver_user_review, columns = ['businessId','nickname','relation','name','address','rating','created'])

    #구글 비지니스 가게의 후기 정보
    df4 = pd.DataFrame(google_store, columns = ['businessId','name','nickname','rating','body','created'])

    #구글 개인과 관련된 가게정보
    df5 = pd.DataFrame(google_user_review,columns = ['businessId','name','nickname','address','rating','created'])

    df.append(df1)
    df.append(df2)
    df.append(df3)
    df.append(df4)
    df.append(df5)

    save_data(df)
    
    logger.info("크롤링 종료!!!")
    return naver_store, stores,naver_user_review, google_store, google_user_review


browser, hasSelenium = setSelenium()
try:
    if hasSelenium is None:
        pass
except Exception as e:
    hasSelenium = False
if hasSelenium is False:
    browser, hasSelenium = setSelenium()
naver_store, stores, naver_user_review, google_store, google_user_review = pipeLined("김천 체험",10,5,5,browser)
#parameter1: query
#parameter2: 상점 갯수
#parameter3: 상점의 리뷰갯수
#parameter4: 개인의 리뷰갯수
#parameter5: selenium browser